In [5]:
# -------------------------------
# Resolution in First-Order Logic
# (with unification, set-of-support)
# -------------------------------

from dataclasses import dataclass
from typing import Dict, Set, Iterable, Optional


# ---------- Term / Literal / Clause structures ----------

@dataclass(frozen=True)
class Var:
    name: str
    def __repr__(self): return self.name

@dataclass(frozen=True)
class Const:
    name: str
    def __repr__(self): return self.name

@dataclass(frozen=True)
class Func:
    name: str
    args: tuple
    def __repr__(self):
        if not self.args:
            return self.name
        return f"{self.name}({', '.join(map(repr, self.args))})"


@dataclass(frozen=True)
class Literal:
    pred: str
    args: tuple        # tuple of terms
    positive: bool = True

    def __repr__(self):
        sign = "" if self.positive else "¬"
        return f"{sign}{self.pred}({', '.join(map(repr, self.args))})"

    def negate(self):
        return Literal(self.pred, self.args, not self.positive)


@dataclass(frozen=True)
class Clause:
    lits: frozenset    # set of Literal

    def __repr__(self):
        if not self.lits:
            return "□"      # empty clause
        return " ∨ ".join(sorted(map(repr, self.lits), key=str))


Subst = Dict[Var, object]   # variable → term


# ---------- Unification ----------

def is_var(t) -> bool:
    return isinstance(t, Var)


def apply_subst_term(t, s: Subst):
    # apply substitution to a term
    if isinstance(t, Var):
        while t in s:
            t = s[t]
        return t
    if isinstance(t, Func):
        return Func(t.name, tuple(apply_subst_term(a, s) for a in t.args))
    return t  # Const


def apply_subst_literal(l: Literal, s: Subst) -> Literal:
    return Literal(l.pred, tuple(apply_subst_term(a, s) for a in l.args), l.positive)


def apply_subst_clause(c: Clause, s: Subst) -> Clause:
    return Clause(frozenset(apply_subst_literal(l, s) for l in c.lits))


def unify(x, y, s: Optional[Subst] = None) -> Optional[Subst]:
    """
    Robinson's unification algorithm (no occurs-check, enough for this example).
    Returns a substitution or None if not unifiable.
    """
    if s is None:
        s = {}
    x = apply_subst_term(x, s)
    y = apply_subst_term(y, s)

    if x == y:
        return s

    if is_var(x):
        s = dict(s)
        s[x] = y
        return s

    if is_var(y):
        s = dict(s)
        s[y] = x
        return s

    if isinstance(x, Func) and isinstance(y, Func) and x.name == y.name and len(x.args) == len(y.args):
        for a, b in zip(x.args, y.args):
            s = unify(a, b, s)
            if s is None:
                return None
        return s

    return None


# ------------------------------
# Better variable renaming
# Produces x1, x2, x3, ...
# ------------------------------

var_counter = 0   # global counter

def standardize_apart(clause: Clause) -> Clause:
    """
    Renames all variables in the clause using a global counter,
    making names like x1, x2, x3 instead of memory addresses.
    """
    global var_counter

    var_map: Dict[Var, Var] = {}

    def rename_term(t):
        global var_counter
        if isinstance(t, Var):
            if t not in var_map:
                var_counter += 1
                var_map[t] = Var(f"{t.name}{var_counter}")
            return var_map[t]
        if isinstance(t, Func):
            return Func(t.name, tuple(rename_term(a) for a in t.args))
        return t

    new_lits = []
    for lit in clause.lits:
        new_args = tuple(rename_term(a) for a in lit.args)
        new_lits.append(Literal(lit.pred, new_args, lit.positive))

    return Clause(frozenset(new_lits))




def resolve(c1: Clause, c2: Clause) -> Set[Clause]:
    """
    Resolve two clauses, returning a set of resolvent clauses.
    """
    resolvents: Set[Clause] = set()

    # standardize variables so clauses don't share them
    c1s = standardize_apart(c1)
    c2s = standardize_apart(c2)


    for l1 in c1s.lits:
        for l2 in c2s.lits:
            # complementary literals with same predicate
            if l1.pred == l2.pred and l1.positive != l2.positive and len(l1.args) == len(l2.args):
                # try to unify their arguments
                s: Optional[Subst] = {}
                for a, b in zip(l1.args, l2.args):
                    s = unify(a, b, s)
                    if s is None:
                        break
                if s is None:
                    continue

                new_lits = (c1s.lits - {l1}) | (c2s.lits - {l2})
                new_clause = apply_subst_clause(Clause(new_lits), s)
                resolvents.add(new_clause)

    return resolvents


# ---------- Resolution algorithm (set of support) ----------

def resolution(kb: Iterable[Clause], negated_goal: Iterable[Clause],
               max_steps: int = 200, verbose: bool = True) -> bool:
    """
    Resolution refutation.
    kb           : set of premise clauses (in CNF)
    negated_goal : clauses for ¬S
    Returns True if contradiction (empty clause) is derived.
    """
    kb = set(kb)
    sos = set(negated_goal)        # set of support
    clauses = kb | sos

    step = 0
    while step < max_steps:
        step += 1
        new: Set[Clause] = set()

        # resolve every SOS clause with every existing clause
        for c in list(sos):
            for d in list(clauses - {c}):
                for resolvent in resolve(c, d):
                    if verbose:
                        print(f"Step {step}: Resolve {c} and {d}  ⇒  {resolvent}")
                    if not resolvent.lits:
                        print("Derived empty clause. Contradiction found.")
                        return True
                    if resolvent not in clauses:
                        new.add(resolvent)

        if not new:
            if verbose:
                print("No new clauses. Resolution failed.")
            return False

        sos = new
        clauses |= new

    if verbose:
        print("Max steps reached. Resolution failed.")
    return False


# Constants
John = Const("John")
Apple = Const("Apple")
Vegetables = Const("Vegetables")
Anil = Const("Anil")
Harry = Const("Harry")
Peanuts = Const("Peanuts")

# Variables
x = Var("x")
y = Var("y")

def L(pred, *args, positive=True):
    """Helper to build a literal quickly."""
    return Literal(pred, args, positive)


# Premises in CNF (already converted):

# 1. John likes all kinds of food.
#    ∀z (Food(z) → Likes(John, z))
#    CNF: ¬Food(z) ∨ Likes(John, z)
c1 = Clause(frozenset({
    L("Food", x, positive=False),
    L("Likes", John, x)
}))

# 2. Apple and vegetables are food.
#    Food(Apple)
#    Food(Vegetables)
c2 = Clause(frozenset({ L("Food", Apple) }))
c3 = Clause(frozenset({ L("Food", Vegetables) }))

# 3. Anything anyone eats and is not killed is food.
#    ∀p∀q ((Eats(p,q) ∧ ¬Killed(p)) → Food(q))
#    CNF: ¬Eats(p,q) ∨ Killed(p) ∨ Food(q)
c4 = Clause(frozenset({
    L("Eats", x, y, positive=False),
    L("Killed", x),
    L("Food", y)
}))

# 4. Anil eats peanuts and is still alive.
#    Eats(Anil, Peanuts)
#    Alive(Anil)
c5 = Clause(frozenset({ L("Eats", Anil, Peanuts) }))
c6 = Clause(frozenset({ L("Alive", Anil) }))

# 5. Harry eats everything that Anil eats.
#    ∀q (Eats(Anil, q) → Eats(Harry, q))
#    CNF: ¬Eats(Anil, q) ∨ Eats(Harry, q)
c7 = Clause(frozenset({
    L("Eats", Anil, y, positive=False),
    L("Eats", Harry, y)
}))

# 6. Anyone who is alive implies not killed.
#    ∀p (Alive(p) → ¬Killed(p))
#    CNF: ¬Alive(p) ∨ ¬Killed(p)
c8 = Clause(frozenset({
    L("Alive", x, positive=False),
    L("Killed", x, positive=False)
}))

# 7. Anyone who is not killed implies alive.
#    ∀p (¬Killed(p) → Alive(p))
#    CNF: Killed(p) ∨ Alive(p)
c9 = Clause(frozenset({
    L("Killed", x),
    L("Alive", x)
}))

kb_clauses = [c1, c2, c3, c4, c5, c6, c7, c8, c9]


# ---------- Goal: John likes peanuts ----------

# We prove it by refutation: add ¬Likes(John, Peanuts) and derive contradiction.
negated_goal_clause = Clause(frozenset({
    L("Likes", John, Peanuts, positive=False)
}))


if __name__ == "__main__":
    print("Knowledge base clauses:")
    for i, c in enumerate(kb_clauses, start=1):
        print(f"{i}: {c}")
    print("\nNegated goal:")
    print(f"¬(John likes peanuts): {negated_goal_clause}\n")

    print("Starting resolution...\n")
    proved = resolution(kb_clauses, [negated_goal_clause], verbose=True)

    if proved:
        print("\nTherefore, by resolution, John likes peanuts.")
    else:
        print("\nCould not prove the goal with resolution.")


Knowledge base clauses:
1: Likes(John, x) ∨ ¬Food(x)
2: Food(Apple)
3: Food(Vegetables)
4: Food(y) ∨ Killed(x) ∨ ¬Eats(x, y)
5: Eats(Anil, Peanuts)
6: Alive(Anil)
7: Eats(Harry, y) ∨ ¬Eats(Anil, y)
8: ¬Alive(x) ∨ ¬Killed(x)
9: Alive(x) ∨ Killed(x)

Negated goal:
¬(John likes peanuts): ¬Likes(John, Peanuts)

Starting resolution...

Step 1: Resolve ¬Likes(John, Peanuts) and Likes(John, x) ∨ ¬Food(x)  ⇒  ¬Food(Peanuts)
Step 2: Resolve ¬Food(Peanuts) and Food(y) ∨ Killed(x) ∨ ¬Eats(x, y)  ⇒  Killed(x12) ∨ ¬Eats(x12, Peanuts)
Step 3: Resolve Killed(x12) ∨ ¬Eats(x12, Peanuts) and ¬Alive(x) ∨ ¬Killed(x)  ⇒  ¬Alive(x16) ∨ ¬Eats(x16, Peanuts)
Step 3: Resolve Killed(x12) ∨ ¬Eats(x12, Peanuts) and Eats(Harry, y) ∨ ¬Eats(Anil, y)  ⇒  Killed(Harry) ∨ ¬Eats(Anil, Peanuts)
Step 3: Resolve Killed(x12) ∨ ¬Eats(x12, Peanuts) and Eats(Anil, Peanuts)  ⇒  Killed(Anil)
Step 4: Resolve ¬Alive(x16) ∨ ¬Eats(x16, Peanuts) and Alive(x) ∨ Killed(x)  ⇒  Killed(x37) ∨ ¬Eats(x37, Peanuts)
Step 4: Resolve ¬Alive(x16)